In [1]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install spotipy

Defaulting to user installation because normal site-packages is not writeable


In [190]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score
from sklearn.metrics import silhouette_score
from sklearn.metrics import classification_report
from sklearn.metrics import pairwise_distances_argmin_min

In [191]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="e5af6275806e4e8ebf14a5d71e202fad",
                                                           client_secret="a72673ed83bb437d83c801b760a066d5"))

In [192]:
def get_df_from_playlist(playlist_id):
    
    results = sp.user_playlist_tracks("spotify", playlist_id)
    tracks = results['items']

    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    track_ids = [track['track']['uri'] for track in tracks]
    
    start = 0
    df = pd.DataFrame()

    for stop in range(0, len(track_ids)+100, 100):
        if start != stop:
            new_df = pd.json_normalize(sp.audio_features(track_ids[start:stop]))
            df = pd.concat([new_df, df])
            start = stop

    df = df.reset_index(drop=True)
    df.drop(['type', 'uri', 'track_href', 'analysis_url',], axis=1, inplace=True)
    
    main_artist_names = [track['track']['artists'][0]['name'] for track in tracks]
    song_names = [track['track']['name'] for track in tracks]
    df['main_artist'] = main_artist_names
    df['song_name'] = song_names

    return df

In [193]:
def get_df_from_multi_playlists(playlists):

    df = pd.DataFrame()

    for pl in playlists:

        df_ = get_df_from_playlist(playlists[pl])
        df = pd.concat([df,df_]).drop_duplicates().reset_index(drop=True)

    return df

In [194]:
#official Billboard Hot 100 playlist

df_hot100_current = get_df_from_playlist("6UeSakyzhiEt4NB3UAd6NQ")

In [195]:
#From Wikipedia: List of Billboard Hot 100 top-ten singles in 2023

#r = requests.get('https://en.wikipedia.org/wiki/List_of_Billboard_Hot_100_top-ten_singles_in_2023')
#r.status_code

In [196]:
#official Spotify top 50 charts: Global and top 10 markets
#https://en.wikipedia.org/wiki/List_of_largest_recorded_music_markets (2022 table)

top_by_market_playlists = {
    
    "top_gb" : "37i9dQZEVXbNG2KDcFcKOF", #Global
    "top_us" : "37i9dQZEVXbLp5XoPON0wI", #United States
    "top_jp" : "37i9dQZEVXbKqiTGXuCOsB", #Japan
    "top_uk" : "37i9dQZEVXbMwmF30ppw50", #United Kingdom
    "top_de" : "37i9dQZEVXbK8BKKMArIyl", #Germany
    "top_fr" : "37i9dQZEVXbIPWwFssbupI", #France
    "top_sk" : "37i9dQZEVXbJZGli0rRP3r", #South Korea
    "top_ca" : "37i9dQZEVXbMda2apknTqH", #Canada
    "top_br" : "37i9dQZEVXbKzoK95AbRy9", #Brazil
    "top_au" : "37i9dQZEVXbK4fwx2r07XW", #Australia
    "top_it" : "37i9dQZEVXbJUPkgaWZcWG" #Italy (no playlist available for China)
}

In [197]:
df_top_all_markets = get_df_from_multi_playlists(top_by_market_playlists)

In [220]:
df_top_songs = pd.concat([df_hot100_current, df_top_all_markets]).drop_duplicates().dropna().reset_index(drop=True)

In [199]:
#most popular music genres
#https://headphonesaddict.com/music-genre-statistics/#Most_Popular_Music_Genre_Statistics
#separate by genres and decades
#top_genres_gb = ['Pop', 'Rock', '90s', '80s', 'Soundtracks', 'Hip-hop/Rap/Trap', 'R&B', 'Dance/Electronic/House', '70s', 'Soul/Blues']

In [200]:
#most popular music genres
#spotify playlists

#"most streamed rap songs on spotify"

In [201]:
#best of playlists
#genre
best_by_genre_playlists = {
    
    "best_hiphop" : "37i9dQZF1DX5qmTk3PVatJ", #greatest hiphop beats of all time - 50
    "best_classic_rock" : "37i9dQZF1DWXRqgorJj26U", #rock classics - 200
    "best_pop" : "7AsZxdanrHJFNGp3q7tp1f" #best of pop 1990 2023 - 1200
}

In [202]:
df_best_by_genre = get_df_from_multi_playlists(best_by_genre_playlists)

In [203]:
# decade
best_by_decade_playlists = {
    
    "best_80s" : "37i9dQZF1DX4UtSsGT1Sbe", #all out 80s - 50
    "best_90s" : "37i9dQZF1DXbTxeAdrVG2l" #all out 90s - 50
}

In [204]:
df_best_by_decade = get_df_from_multi_playlists(best_by_decade_playlists)

In [221]:
#concat all songs
df_all_best_songs = pd.concat([df_best_by_genre, df_best_by_decade]).drop_duplicates().dropna().reset_index(drop=True)

In [206]:
#df_all_best_filtered = pd.merge(df_all_best_songs, df_top_songs, how='outer', indicator=True)
#df_all_best_filtered = df_all_best_filtered[df_all_best_filtered['_merge']=='left_only']

In [207]:
cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

## II

In [231]:
def apply_kmeans(scaled_x, cluster, df):
    x = df[cols]
    scaler = StandardScaler()
    x_prep = scaler.fit_transform(x)

    kmeans = KMeans(n_clusters=3, random_state=42)
    kmeans.fit(x_prep)

    clusters = kmeans.predict(x_prep)

    scaled_df = pd.DataFrame(x_prep, columns=x.columns)
    scaled_df['song_name'] = df['song_name']
    scaled_df['main_artist'] = df['main_artist']
    scaled_df['cluster'] = clusters
    
    filtered_df = np.array(scaled_df[scaled_df['cluster'] == cluster[0]][x.columns], order="C")
    closest, _ = pairwise_distances_argmin_min(scaled_x, filtered_df)
    print('\n[RECOMMENDED SONG]')
    print(' - '.join([scaled_df.loc[closest]['song_name'].values[0], scaled_df.loc[closest]['main_artist'].values[0]]))

## III

In [232]:

#format: "song title" OR "song tile" + "artist"

In [233]:
def get_song_by_keywords(kws):
    
    results = sp.search(q=f'track:{kws}', limit=1)
    
    track_id = results['tracks']['items'][0]['id']
    track_name = results['tracks']['items'][0]['name']
    track_artist = results['tracks']['items'][0]['artists'][0]['name']
    
    print(track_name, '-', track_artist)
    
    audio_features = sp.audio_features(track_id)
    df_ = pd.DataFrame(audio_features)
    new_features = df_[x.columns]
    scaled_x = scaler.transform(new_features)
    cluster = kmeans.predict(scaled_x)
    
    df_clusters = df_top_songs if track_id in df_top_songs['id'].values else df_all_best_songs
    
    apply_kmeans(scaled_x, cluster, df_clusters)

In [235]:
#kws = input('Choose a song: ')
kws = 'viva la vida'
get_song_by_keywords(kws)

Viva La Vida - Coldplay


C:\Users\grwc9\AppData\Roaming\Python\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(



[RECOMMENDED SONG]
I'm a Slave 4 U - Britney Spears
